Lo script seguente itera in ordine alfabetico i file inseriti nella cartella "files" e, a seconda del tipo, li sposta nella relativa sottocartella. Se la sottocartella non esiste lo script la crea automaticamente. Viene anche creato un file "recap.csv" in cui ogni volta che viene spostato un file viene aggiornato con il relativo nome, tipo e dimensione.

In [1]:
import os
import shutil
import csv

In [2]:
#mi sposto nella cartella files
os.chdir('files')

In [21]:
#Creo liste formati accettati
img_exts = [".jpg",".jpeg",".png"]
doc_exts = [".odt",".txt"]
audio_exts = [".mp3"]

In [22]:
#se il file di recap non esiste creo il file di recap, se esiste già lo apro
if not os.path.exists('recap.csv'):
    recap = open('recap.csv', 'w', newline='')#lo creo: 'w' sta per 'write'
    writer = csv.writer(recap)
    writer.writerow(['name', 'type', 'size (B)'])#scrivo header del csv
else:
    recap = open('recap.csv', 'a', newline='')#lo apro in 'append' con 'a'
    writer = csv.writer(recap)

In [23]:
#creo le sottocartelle
for dirname in ['audio', 'docs', 'images']:
    if not os.path.exists(dirname):#...se queste non esistono
        os.makedirs(dirname)
#itero dentro cartella corrente (cioè files: mi sono spostato dentro all'inizio)
for f in sorted(os.listdir('.')):
    
    #separo il nome del file dall'estensione
    name, ext = os.path.splitext(f)
    
    if ext in img_exts:#check estensione
        size = os.path.getsize(f)#calcolo dimensione in byte
        print(f'{name} type:image size:{size}B')#stampo info
        writer.writerow([name, 'image', size])#aggiorno recap
        shutil.move(f, 'images/')#sposto in sottocartella
        
    elif ext in doc_exts:
        size = os.path.getsize(f)
        print(f'{name} type:doc size:{size}B')
        writer.writerow([name, 'doc', size])
        shutil.move(f, 'docs/')
        
    elif ext in audio_exts:
        size = os.path.getsize(f)
        print(f'{name} type:audio size:{size}B')
        writer.writerow([name, 'audio', size])
        shutil.move(f, 'audio/')
        
    else:
        print('Formato non riconosciuto.')
    
recap.close()

Formato non riconosciuto.
bw type:image size:94926B
ciao type:doc size:12B
daffodil type:image size:24657B
Formato non riconosciuto.
eclipse type:image size:64243B
Formato non riconosciuto.
pippo type:doc size:8299B
Formato non riconosciuto.
song1 type:audio size:1087849B
song2 type:audio size:764176B
trump type:image size:10195B


Il seguente script itera invece sulla cartella immagini e costruisce una tabella riassuntiva contentente le informazioni di colore dei file.

In [24]:
from PIL import Image
import numpy as np
from tabulate import tabulate

In [33]:
#Itero sulla cartella "images" usando "glob.glob" che mi restituisce tutti i file presenti sulla cartella(path compreso)
#Nello stesso ciclo for apro ogni file tramite "Image.open" e li trasformo in array
#Ottengo il nome di ogni file che inserirò nelle informazioni richieste da stampare

list_info = []

for f in sorted(os.listdir('images')):
    nome = os.path.splitext(f)[0] 
    img = Image.open(os.path.join('images', f))#uso join per costruire nome file relativo a sottocartella
    im_array = np.array(img)

#Creo un dizionario che ha come chiavi tutti i dati che servono da stampare
#Tramite "im_array.shape" ottengo altezza e larghezza di ogni immagine
#Se l'array è bidimensionale faccio la media di tutti i valori dell'array e ottengo il valore della scala di grigi
#Se l'array è tridimensionale faccio la media di tutti i valori dell'array unendoli in una tupla e ottengo i valori 'R,G,B'
#Se l'array è tridimensionale ed ha 4 livelli di colore ottengo anche il canale 'ALPHA'
#Aggiungo tutti i dati ad una lista precedentemente creata

    
    im_info = {'name': nome,'height':0,'width':0,'grayscale':0,'R':0,'G':0,'B':0,'ALPHA':0}
    im_info['height'] = im_array.shape[0]
    im_info['width']=im_array.shape[1]
    if im_array.ndim == 2:
        im_info['grayscale'] = np.mean(im_array)
    elif im_array.ndim == 3:
        im_info['R'] = np.mean(im_array,axis = (0,1))[0]
        im_info['G'] = np.mean(im_array,axis = (0,1))[1]
        im_info['B'] = np.mean(im_array,axis = (0,1))[2]
        if im_array.shape[2] == 4:
            im_info['ALPHA'] = np.mean(im_array,axis = (0,1))[3]
    else:
        print("Non è un'immagine")
        
    list_info.append(im_info)

#Creo una tabella tramite la libreria "tabulate" inserendo la lista di dizionari creata precedentemente
#La tabella avrà come colonne le chiavi dei dizionari

    

print(tabulate(list_info, floatfmt=".2f",tablefmt='fancy_grid',headers = "keys"))

╒══════════╤══════════╤═════════╤═════════════╤════════╤════════╤═══════╤═════════╕
│ name     │   height │   width │   grayscale │      R │      G │     B │   ALPHA │
╞══════════╪══════════╪═════════╪═════════════╪════════╪════════╪═══════╪═════════╡
│ bw       │      512 │     512 │       21.48 │   0.00 │   0.00 │  0.00 │    0.00 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ daffodil │      500 │     335 │        0.00 │ 109.23 │  85.52 │  4.77 │    0.00 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ eclipse  │      256 │     256 │        0.00 │ 109.05 │ 109.52 │ 39.85 │  133.59 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ trump    │      183 │     275 │        0.00 │  97.01 │  98.99 │ 90.92 │    0.00 │
╘══════════╧══════════╧═════════╧═════════════╧════════╧════════╧═══════╧═════════╛
